<a href="https://colab.research.google.com/github/mzwing/AI-related/blob/master/notebooks/stable-diffusion.cpp/Anything_LCM_GGUF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U --force-reinstall huggingface_hub[hf_transfer]

In [ ]:
# prepare
!rm -rf sample_data
!mkdir -p Anything-LCM-GGUF

In [ ]:
!huggingface-cli login

In [ ]:
# get original model
!sudo apt-get install aria2 -y
!mkdir -p Anything-LCM
!echo -e "https://huggingface.co/mzwing/Anything-LCM-GGUF/resolve/main/anything_v30-LCM.safetensors?download=true\n out=anything_v30-LCM.safetensors" > download.txt
!aria2c -c -x16 -d Anything-LCM --input-file=download.txt
!rm -rf download.txt

In [ ]:
!sudo apt-get install aria2 -y
!aria2c -c -x16 -d Anything-LCM-GGUF https://huggingface.co/mzwing/Anything-LCM-GGUF/resolve/main/anything_v30-LCM.q8_0.gguf?download=true -o anything_v30-LCM.q8_0.gguf

In [ ]:
# get compile result (CPU)
!aria2c -c -x16 https://github.com/MZWNET/actions/releases/download/sd-master-48bcce4/sd-master-48bcce4-bin-linux-avx2-openblas-x64.zip
!unzip sd-master-48bcce4-bin-linux-avx2-cublas-cu121-x64.zip -d .
!rm -rf sd-master-48bcce4-bin-linux-avx2-cublas-cu121-x64.zip

In [ ]:
# prepare for GPU
!sudo apt-get install nvidia-cuda-toolkit -y

In [ ]:
# get compile result (GPU)
!aria2c -c -x16 https://github.com/MZWNET/actions/releases/download/sd-master-48bcce4/sd-master-48bcce4-bin-linux-avx2-cublas-cu121-x64.zip
!unzip sd-master-48bcce4-bin-linux-avx2-cublas-cu121-x64.zip -d .
!rm -rf sd-master-48bcce4-bin-linux-avx2-cublas-cu121-x64.zip

In [ ]:
# quantize
import concurrent.futures
import os
from tqdm import tqdm

parameters = [ "f16", "q8_0", "q5_1", "q5_0", "q4_1", "q4_0" ]
models = [ "anything_v30-LCM.safetensors" ]

tasks = [(model, param) for model in models for param in parameters]

def run_command(task):
    model, param = task
    model_name = model.split('.')[0]
    os.system(f"./sd -M convert -m Anything-LCM/{model} -o Anything-LCM-GGUF/{model_name}.{param}.gguf --type {param}")

with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
    list(tqdm(executor.map(run_command, tasks), total=len(tasks)))

In [ ]:
# test
!./sd -m Anything-LCM/anything_v30-LCM.safetensors -p "1girl, beautiful, school-uniform, nsfw" --cfg-scale 1 --steps 4 -s -1 -v -o test.png

In [ ]:
# test
!./sd -m Anything-LCM-GGUF/anything_v30-LCM.q8_0.gguf -p "1girl, beautiful, school-uniform, nsfw" --cfg-scale 1 --steps 4 -s -1 -v -o test.png

In [ ]:
!HF_HUB_ENABLE_HF_TRANSFER=1 huggingface-cli upload --repo-type model --commit-message "GGUF model commit (made with stable-diffusion.cpp commit 583cc5b)" Anything-LCM-GGUF ./Anything-LCM-GGUF